<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-gwkcareer/blob/main/Chapter2_1_%EA%B3%BC%EC%A0%9CB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제 B. 주어진 문장에서 나올 다음 단어를 예측하는 모델 구현

- Last word prediction dataset 준비
    - 기존의 IMDB dataset을 그대로 활용합니다.
    - `collate_fn` 함수에 다음 수정사항들을 반영하면 됩니다.
        - Label은 text를 token으로 변환했을 때 마지막 token의 id로 설정합니다.
        - 입력 data는 마지막 token을 제외한 나머지 token들의 list로 설정합니다.
    - `from torch.nn.utils.rnn import pad_sequence`를 import해서 사용하셔도 좋습니다.
    - Truncation은 기존과 똑같이 진행하시면 됩니다.
- [ ]  Loss function 및 classifier output 변경
    - 마지막 token id를 예측하는 것이기 때문에 binary classification이 아닌 일반적인 classification 문제로 바뀝니다. MNIST 과제에서 했던 것 처럼 `nn.CrossEntropy` loss와 `TextClassifier`의 출력 차원을 잘 조정하여 task를 풀 수 있도록 수정하시면 됩니다.
- [ ]  학습 결과 report
    - 과제 A에서 사용한 것과 동일한 모델로 last word prediction을 학습하시면 됩니다.

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install sentencepiece sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 23.2 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from torch.nn.utils.rnn import pad_sequence
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:

    #texts.append(row['text'])
    #labels.append(row['label']

    # Tokenize the text with truncation and padding
    tokenized = tokenizer(row['text'], padding=True, truncation=True, max_length=max_len).input_ids

    # Last token as the label
    labels.append(tokenized[-2])

    # Input data excluding the last token
    texts.append(tokenized[:-2])

  #texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)

  # Padding the tokenized input sequences
  texts = [torch.tensor(t) for t in texts]
  texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)

  labels = torch.LongTensor(labels)
  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## MultiHeadAttention

In [ ]:
from torch import nn
from math import sqrt

class MultiHeadAttention(nn.Module):
  def __init__(self, input_dim, d_model, n_heads):
    super().__init__()

    assert d_model % n_heads == 0, "d_model은 n_heads로 나눠떨어져야 합니다."

    self.input_dim = input_dim
    self.d_model = d_model
    self.n_heads = n_heads
    self.d_head = d_model // n_heads  # 각 헤드에서의 차원 D' = D / H

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):

    batch_size, seq_len, _ = x.size()

    q, k, v = self.wq(x), self.wk(x), self.wv(x)

    # 2. Q, K, V를 여러 헤드로 분리 (view를 사용해 [batch_size, seq_len, n_heads, d_head]로 변경)
    q = q.view(batch_size, seq_len, self.n_heads, self.d_head)
    k = k.view(batch_size, seq_len, self.n_heads, self.d_head)
    v = v.view(batch_size, seq_len, self.n_heads, self.d_head)

    # 3. 헤드 차원과 시퀀스 길이를 교환 (transpose로 차원 변환: [batch_size, n_heads, seq_len, d_head])
    q = q.transpose(1, 2)
    k = k.transpose(1, 2)
    v = v.transpose(1, 2)

    # 4. Scaled Dot-Product Attention 계산 (Q * K^T / sqrt(d_head))
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_head)

    if mask is not None:
       mask = mask.unsqueeze(1)
       score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)

    # 8. 헤드를 다시 합치기 (transpose와 view를 사용해 [batch_size, seq_len, d_model]로 변환)
    result = result.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)

    result = self.dense(result)

    return result

In [ ]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff, n_heads, dropout_rate=0.1):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    # Multi-Head Attention 변경
    self.mha = MultiHeadAttention(input_dim, d_model, n_heads)

    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

     # Dropout 및 Layer Normalization 설정
    self.dropout1 = nn.Dropout(dropout_rate)
    self.dropout2 = nn.Dropout()
    self.layernorm1 = nn.LayerNorm(d_model)
    self.layernorm2 = nn.LayerNorm(d_model)

  def forward(self, x, mask):
    # Multi-Head Attention (MHA)
    x1 = self.mha(x, mask)
    x1 = self.dropout1(x1)
    x1 = self.layernorm1(x1 + x)

    # Feed-Forward Network (FFN)
    x2 = self.ffn(x1)
    x2 = self.dropout2(x2)
    x2 = self.layernorm2(x2 + x1)

    return x2



## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [ ]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


In [ ]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff, n_heads, dropout_rate=0.1):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff, n_heads) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, -1] #	마지막 단어만 예측하는 경우라면, x[:, -1]을 사용할 수 있습니다.
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 5, 32, 4)

## 학습

In [ ]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')

#손실 함수 정의 - CrossEntropyLoss
#loss_fn = nn.BCEWithLogitsLoss()
loss_fn  = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1)  # 다중클래스
    #preds = (preds > 0).long()[..., 0]  # 이진클래스

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [ ]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    #preds = model(inputs)[..., 0]
    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 1541.6411756277084
=========> Train acc: 0.563 | Test acc: 0.565
Epoch   1 | Train Loss: 1028.9833207130432
=========> Train acc: 0.582 | Test acc: 0.583
Epoch   2 | Train Loss: 970.948743224144
=========> Train acc: 0.586 | Test acc: 0.582
Epoch   3 | Train Loss: 932.510821223259
=========> Train acc: 0.592 | Test acc: 0.585
Epoch   4 | Train Loss: 894.5500869750977
=========> Train acc: 0.596 | Test acc: 0.587
Epoch   5 | Train Loss: 853.9401426315308
=========> Train acc: 0.606 | Test acc: 0.589
Epoch   6 | Train Loss: 814.407829284668
=========> Train acc: 0.616 | Test acc: 0.584
Epoch   7 | Train Loss: 770.6868109703064
=========> Train acc: 0.627 | Test acc: 0.584
Epoch   8 | Train Loss: 723.8752086162567
=========> Train acc: 0.649 | Test acc: 0.582
Epoch   9 | Train Loss: 678.7488462924957
=========> Train acc: 0.659 | Test acc: 0.582
Epoch  10 | Train Loss: 630.0777869224548
=========> Train acc: 0.704 | Test acc: 0.563
Epoch  11 | Train Loss: 581.80793